In [1]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [7]:
base_path = "/Users/Bratz/ML_Projects/nlp-with-rnn" 
filename = "aot_script.txt"
path_to_file = os.path.join(base_path, filename)

In [8]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 146486 characters


In [9]:
# Take a look at the first 250 characters in text
print(text[:250])

﻿Armin
(Narration) On that day, humanity remembered… the fear of oppression under them.

(Narration) And we were disgraced to live in these cages we called walls.
[Scene: On the battlefield of Titans.]
Shadis
Everyone, get ready to fight! There


In [10]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [11]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: ﻿Armin
(Narr
Encoded: [79 22 66 61 57 62  1  0  6 35 49 66 66]


In [12]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

﻿Armin
(Narr


In [13]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [14]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [15]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [16]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
﻿Armin
(Narration) On that day, humanity remembered… the fear of oppression under them.

(Narrati

OUTPUT
Armin
(Narration) On that day, humanity remembered… the fear of oppression under them.

(Narratio


EXAMPLE

INPUT
n) And we were disgraced to live in these cages we called walls.
[Scene: On the battlefield of Tita

OUTPUT
) And we were disgraced to live in these cages we called walls.
[Scene: On the battlefield of Titan


In [17]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           20480     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 80)            82000     
Total params: 5,349,456
Trainable params: 5,349,456
Non-trainable params: 0
_________________________________________________________________


In [19]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 80) # (batch_size, sequence_length, vocab_size)


In [20]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-1.6801679e-03 -9.1022695e-04 -3.7521524e-03 ... -3.0519857e-04
    2.4130316e-03  6.0018557e-03]
  [-4.4431287e-04  4.2923545e-03 -6.8619387e-04 ... -4.4490267e-03
    6.6724680e-03  9.1967117e-03]
  [-3.0763452e-03  7.5744116e-03 -9.2594960e-04 ... -3.0065151e-03
    9.4967745e-03  7.6483679e-03]
  ...
  [-6.2802741e-03 -7.2201947e-06 -5.7600546e-03 ... -4.7483733e-03
    3.5234271e-03  1.0686280e-02]
  [-8.6538438e-03 -4.5345398e-03 -2.6903125e-03 ... -9.0417573e-03
   -7.4237841e-04  9.8694861e-03]
  [ 2.5310821e-03 -8.7333778e-03 -3.6252548e-03 ... -7.8472793e-03
    1.5347449e-03  1.0691929e-02]]

 [[-3.6126634e-03  2.0606392e-03 -1.0982529e-03 ... -4.4172280e-05
    4.6404293e-03 -7.9518434e-04]
  [-5.7658344e-03 -2.3434777e-03  1.9416496e-03 ...  1.7526583e-04
    5.8665103e-03 -6.8390742e-03]
  [-5.8797565e-03 -2.5252926e-03 -3.1793579e-03 ...  2.9028306e-04
    7.4480651e-03  1.4002734e-03]
  ...
  [-4.6969960e-03  2.9260004e-03  6.7600450e-03 ... -1.3013126e

In [21]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-1.6801679e-03 -9.1022695e-04 -3.7521524e-03 ... -3.0519857e-04
   2.4130316e-03  6.0018557e-03]
 [-4.4431287e-04  4.2923545e-03 -6.8619387e-04 ... -4.4490267e-03
   6.6724680e-03  9.1967117e-03]
 [-3.0763452e-03  7.5744116e-03 -9.2594960e-04 ... -3.0065151e-03
   9.4967745e-03  7.6483679e-03]
 ...
 [-6.2802741e-03 -7.2201947e-06 -5.7600546e-03 ... -4.7483733e-03
   3.5234271e-03  1.0686280e-02]
 [-8.6538438e-03 -4.5345398e-03 -2.6903125e-03 ... -9.0417573e-03
  -7.4237841e-04  9.8694861e-03]
 [ 2.5310821e-03 -8.7333778e-03 -3.6252548e-03 ... -7.8472793e-03
   1.5347449e-03  1.0691929e-02]], shape=(100, 80), dtype=float32)


In [22]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

80
tf.Tensor(
[-1.6801679e-03 -9.1022695e-04 -3.7521524e-03 -8.5485523e-04
 -7.0624449e-03  5.2872775e-03  4.9120840e-03  2.0191390e-03
 -5.6195672e-04 -6.4652693e-03 -5.6369947e-03  5.3611943e-03
 -2.1593465e-04  1.8121543e-03  1.9406830e-03 -1.2919165e-03
 -2.8582334e-03  9.6266414e-04 -4.3558166e-03  2.7931621e-04
  1.9411718e-03  4.9666190e-03  4.0774705e-04 -2.1237261e-03
  3.8379696e-03 -4.5118206e-03 -3.5248528e-04  1.8764834e-03
  6.2487558e-03  2.1883990e-03  2.4018032e-03  2.3137827e-03
 -1.8287571e-03  1.5460881e-03  5.0589570e-04  7.5008525e-03
  7.0730373e-03 -2.0693685e-03 -3.3818623e-03 -3.7954858e-05
  2.5078389e-03 -3.4849383e-03 -3.1522452e-03 -2.8533936e-03
  4.2797445e-04  3.6807866e-03 -6.4006671e-03 -5.7912432e-05
 -2.4317927e-03  1.2152463e-03  2.3144931e-03  8.1563881e-04
 -1.5886774e-03 -2.6188688e-03  6.3005544e-05  3.3068736e-04
  2.5976221e-03 -1.6374866e-04  3.6626286e-03 -7.2324154e-05
 -1.5779483e-03  7.7555329e-04 -1.3044879e-03  1.5465483e-03
  4.331855

In [23]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"WEP’M!…h?aa%D]MqyCKls)’Ec%m'x5.n3DssiHDcK\ufeffTre’[SU%n]I’JRgR’S)pOH0uT(C40zkL8,k)Kl4%Q'e“U4neZB’Zm “C]?"

In [24]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [25]:
model.compile(optimizer='adam', loss=loss)

In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [27]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
22/22 [==============================] - 39s 2s/step - loss: 3.4721
Epoch 2/50
22/22 [==============================] - 42s 2s/step - loss: 3.1126
Epoch 3/50
22/22 [==============================] - 40s 2s/step - loss: 2.6112
Epoch 4/50
22/22 [==============================] - 41s 2s/step - loss: 2.2994
Epoch 5/50
22/22 [==============================] - 40s 2s/step - loss: 2.1211
Epoch 6/50
22/22 [==============================] - 36s 2s/step - loss: 1.9818
Epoch 7/50
22/22 [==============================] - 37s 2s/step - loss: 1.8698
Epoch 8/50
22/22 [==============================] - 36s 2s/step - loss: 1.7828
Epoch 9/50
22/22 [==============================] - 36s 2s/step - loss: 1.7019
Epoch 10/50
22/22 [==============================] - 38s 2s/step - loss: 1.6390
Epoch 11/50
22/22 [==============================] - 37s 2s/step - loss: 1.5735
Epoch 12/50
22/22 [==============================] - 36s 2s/step - loss: 1.5169
Epoch 13/50
22/22 [==============================

In [28]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))


In [30]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [32]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: kumandan
kumandany surve. St’ll all shantes your mom that with me and turns to me!

I’m going back, more south District!
Shadis
Sasha ush… I’m alright.

There’s only one talking your mom…… 
Sh… Hahaard Pixis!

Eren...

[Scene: Sometime later, at night.]
Franz
What… No! It’s that?

Was happle an eaten protection.

(Narration) The ingenuity of these few raised the survival aroming before shomaget in front of Titans are and it… Armin shede out of has all yever nested. We’ve che fundieded wear havades the next sacr, did!
Silding with my way prepared to the building.
Armin
Hey, but even such firepower cluines.

The parrison of mankind once again. What are you doing?!
Eren
We’re withard wather 1
Damis, just like a contion: But the stambs splan ofrranished can ver a chance to gut spoitin

